In [159]:
import torch


def print_config(config):
    attrs = {}
    attrs.update(config.__dict__)

    for key, value in config.__class__.__dict__.items():
        if not key.startswith('__') and key not in attrs:
            attrs[key] = value

    for key, value in attrs.items():
        print(f"{key}: {value}")


class Config:
    # Model
    model_name = "tabnet.86.repro.stopping.at.70"
    input_dim = 6
    target_dim = 1

    # Device & reproducibility
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 5274

    # Optimization
    optimizer_name = "AdamW"
    lr = 1e-4
    weight_decay = 1e-3

    batch_size = 32  # // grad_accum_steps

    # Training schedule
    num_epochs = 100
    scheduler_name = "default"

    # Regularization
    dropout = 0.0
    drop_path_rate = 0.0
    label_smoothing = 0.0

    # Experiment tracking
    neptune_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
    with_id = ""
    resume = False

config = Config()

In [160]:
import random
import numpy as np


def setup_reproducibility(config):
    random.seed(config.seed)
    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(config.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility(config)

In [161]:
import os

PATH = "/Users/arbaaz/Downloads/break/ds"
inputs_path = PATH+"/X_train.parquet"
targets_path = PATH+"/y_train.parquet"
os.listdir(PATH)

['y_train.parquet',
 'X_train.parquet',
 '.gitignore',
 'X_test.reduced.parquet',
 'y_test.reduced.parquet']

In [162]:
import pandas as pd

def load_parquet(path):
    return pd.read_parquet(path)

inputs_df = load_parquet(inputs_path)
targets_df = load_parquet(targets_path)

In [165]:
from sklearn.model_selection import train_test_split

def split(inputs_df, targets_df, seed):
    targets_df = targets_df.reset_index()

    train_ids, test_ids = train_test_split(
        targets_df['id'],                     
        test_size=0.2,
        random_state=seed,
        stratify=targets_df['structural_breakpoint']
    )

    train_inputs = inputs_df.loc[inputs_df.index.get_level_values('id').isin(train_ids)].copy()
    eval_inputs = inputs_df.loc[inputs_df.index.get_level_values('id').isin(test_ids)].copy()

    train_targets = targets_df[targets_df['id'].isin(train_ids)].set_index('id').copy()
    eval_targets = targets_df[targets_df['id'].isin(test_ids)].set_index('id').copy()
    
    return (
        train_inputs,
        train_targets, 
        eval_inputs,
        eval_targets
    )
    
data = split(inputs_df, targets_df, 1)

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np

class SequentialDataset(Dataset):
    def __init__(
        self, 
        inputs_df, 
        target_df,
        mean=None,
        std=None,
        min=None,
        max=None,
        minmax=False,
        zscore=False,
        dimension=2048
    ):  
        inputs = []    
        for id, seq in inputs_df.groupby("id"):
            value = seq["value"].values.astype(np.float32)
            period = seq["period"].values.astype(np.float32)
            input = torch.tensor((value, period)).unsqueeze(0)
            input = F.interpolate(input, size=dimension, mode="nearest-exact").squeeze(0)
            inputs.append(input) # 2, SEQ LEN
            
        targets = []
        for id, target in target_df.groupby("id"):
            target = target["structural_breakpoint"].values.astype(np.int32)
            targets.append(torch.tensor(target))
            
        inputs = torch.stack(inputs)
        targets = torch.stack(targets)    
            
        if zscore:
            inputs = self.perform_zscore(inputs, mean, std)
        if minmax:
            inputs = self.perform_minmax(inputs, min, max)
            
        assert len(inputs) == len(targets), "Length Error"
        self.inputs = inputs
        self.targets = targets
            
    def perform_zscore(self, inputs, mean, std):
        for i in range(len(inputs)):
            inputs[i][0] = (inputs[i][0] - mean) / std
        return inputs

    def perform_minmax(self, inputs, min, max):
        for i in range(len(inputs)):
            inputs[i][0] = (inputs[i][0] - min) / (max - min + 1e-8)
        return inputs
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):   
        return self.inputs[index], self.targets[index]

x = data[0]["values"]
mean, std = x.mean(), x.std()
min, max =  x.min(), x.max()
print(min, max, mean, std)

train_ds = SequentialDataset(data[0], data[1], mean, std, zscore=True, dimension=2048)
eval_ds = SequentialDataset(data[2], data[3], mean, std, zscore=True, dimension=2048)

In [ ]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
train_dl = build_loader(
    config.seed,
    train_ds,
    train=True,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

eval_dl = build_loader(
    config.seed,
    eval_ds,
    train=False,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

-2.2848206626491874 412.5 0.0005905449881844865 0.1057822762501774


In [151]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
train_dl = build_loader(
    config.seed,
    train_ds,
    train=True,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

eval_dl = build_loader(
    config.seed,
    eval_ds,
    train=False,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

In [152]:
for id, (i, t), in enumerate(train_dl):
    print(i[0].mean(), i[0].std())
    if id == 10:
        break

tensor(0.1853) tensor(0.8084)
tensor(0.2097) tensor(0.8180)
tensor(0.0997) tensor(0.7614)
tensor(0.1281) tensor(0.8016)
tensor(0.1781) tensor(0.7859)
tensor(0.1018) tensor(0.7653)
tensor(0.0919) tensor(0.7755)
tensor(0.1288) tensor(0.7872)
tensor(0.1745) tensor(0.7974)
tensor(0.1672) tensor(0.8019)
tensor(0.1730) tensor(0.8099)


/Users/arbaaz/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [154]:
i[0]

tensor([[-0.8772, -0.4693, -4.8250,  ...,  0.0768,  0.5067,  1.3075],
        [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  1.0000,  1.0000]])